## Selecting Rows with Core or ORM

For both Core and ORM, the `select()` function generates a `Select construct` which is used for all `SELECT` queries. Passed to methods like _`Connection.execute()` in Core_ and _`Session.execute()` in ORM_, a `SELECT statement` is emitted in the _current transaction_ and the result rows available via the returned `Result` object.

#### Initialize tables with some data

In [1]:
from sqlalchemy import (
    MetaData, Table, Column, Integer, String, ForeignKey,
    create_engine, insert, select, bindparam, func, cast,
)
from sqlalchemy.orm import Session, registry, relationship

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj = MetaData()

In [3]:
user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

2022-09-25 09:38:07,622 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:07,623 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-25 09:38:07,624 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-25 09:38:07,626 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-25 09:38:07,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-25 09:38:07,629 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-25 09:38:07,630 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-25 09:38:07,631 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-25 09:38:07,631 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-25 09:38:07,633 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-25 09:38:07,634 INFO sqlalchemy.engine.Engine [no key 0.00103s] ()
2022-09-25 09:38:07,638 INFO sqlalchemy.engine.Engine 
C

In [4]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-09-25 09:38:07,752 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:07,754 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-25 09:38:07,755 INFO sqlalchemy.engine.Engine [generated in 0.00274s] ('spongebob', 'Spongebob Squarepants')
2022-09-25 09:38:07,758 INFO sqlalchemy.engine.Engine COMMIT
2022-09-25 09:38:07,759 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:07,761 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-25 09:38:07,763 INFO sqlalchemy.engine.Engine [generated in 0.00360s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-09-25 09:38:07,765 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam("username")).
    scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()

2022-09-25 09:38:07,857 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:07,858 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-09-25 09:38:07,860 INFO sqlalchemy.engine.Engine [generated in 0.00295s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-09-25 09:38:07,862 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
mapped_registry = registry()
Base = mapped_registry.generate_base()

In [7]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [8]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [9]:
mapped_registry.metadata.create_all(engine)

2022-09-25 09:38:08,224 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:08,226 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-25 09:38:08,227 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-25 09:38:08,229 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-25 09:38:08,230 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-25 09:38:08,233 INFO sqlalchemy.engine.Engine COMMIT


#### The `select()` SQL Expression Construct

The `select()` construct builds up a statement in the same way as that of `insert()`, using a _generative approach_ where each method builds more state onto the object. Like the other SQL constructs, it can be _stringified_ in place.

In [10]:
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


Also in the same manner as all other statement-level `SQL constructs`, _to actually run the statement_ we pass it to an `execution` method. Since a `SELECT statement` returns _rows_ we can always iterate the result object to get _Row_ objects back.

In [11]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-09-25 09:38:08,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:08,452 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-25 09:38:08,453 INFO sqlalchemy.engine.Engine [generated in 0.00291s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-09-25 09:38:08,455 INFO sqlalchemy.engine.Engine ROLLBACK


When using the ORM, particularly with a `select()` construct that's _composed against ORM entities_, we will want to execute it using the `Session.execute()` method on the `Session`; using this approach, we continue to get `Row` objects from the result, however these rows are now _capable of including **complete entities**_, such as instances of the `User` class, as individual elements within each row.

In [12]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-09-25 09:38:08,539 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:08,546 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-25 09:38:08,547 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-09-25 09:38:08,549 INFO sqlalchemy.engine.Engine ROLLBACK


While the SQL generated in these examples looks the same whether we invoke `select(user_table)` or `select(User)`, in the more general case they _do not necessarily render the same thing_, as an ORM-mapped class may be mapped to other kinds of `"selectables"` besides tables. __The `select()` that's against an ORM entity also indicates that ORM-mapped instances should be returned in a result, which is not the case when SELECTing from a Table object.__

#### Setting the COLUMNS and FROM clause

The `select()` function accepts positional elements representing any number of `Column` and/or `Table` expressions, as well as a _wide range of compatible objects_, which are resolved into a __list of SQL expressions__ to be `SELECT`ed from that will be returned as columns in the result set. These elements also serve in simpler cases to create the `FROM` clause, which is inferred from the _columns and table-like expressions_ passed.

In [13]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


To `SELECT` from individual columns using a Core approach, `Column` objects are accessed from the _`Table.c` accessor_ and __can be sent directly__; the `FROM` clause will be inferred as the set of all `Table` and other `FromClause` objects that are represented by those columns.

In [14]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


##### Selecting ORM Entities and Columns

_ORM entities_, such our `User class` as well as the _column-mapped attributes upon it_ such as `User.name`, also participate in the `SQL Expression Language system` representing __tables and columns__. Below illustrates an example of `SELECT`ing from the `User` entity, which ultimately renders in the same way as if we had used `user_table` directly.

In [15]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


When executing a statement like the above using the _ORM `Session.execute()` method_, there is __an important difference__ when we _select from a full entity such as `User`, as opposed to `user_table`_, which is that the `entity` itself is returned as _a single element_ within each row. That is, when we fetch rows from the above statement, as there is only the User entity in the list of things to fetch, we get back `Row` objects that have only one element, which contain _instances of the `User` class_.

In [16]:
row = session.execute(select(User)).first()
print(row)

2022-09-25 09:38:08,930 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:08,933 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-09-25 09:38:08,935 INFO sqlalchemy.engine.Engine [generated in 0.00170s] ()
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)


The above `Row` has just one element, representing the `User` entity.

In [17]:
print(row[0])

User(id=1, name='spongebob', fullname='Spongebob Squarepants')


A highly recommended convenience method of achieving the same result as above is to use the `Session.scalars()` method to _execute the statement **directly**_; this method will return a `ScalarResult` object that delivers the _first `column` of each row at once_, in this case, _instances of the `User` class_.

In [18]:
user = session.scalars(select(User)).first()
print(user)

2022-09-25 09:38:09,095 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-09-25 09:38:09,096 INFO sqlalchemy.engine.Engine [cached since 0.1636s ago] ()
User(id=1, name='spongebob', fullname='Spongebob Squarepants')


Alternatively, we can _select individual columns_ of an ORM entity as `distinct` elements within result rows, by _using the class-bound attributes_; when these are passed to a construct such as `select()`, they are resolved into the `Column` or other SQL expression represented by each attribute.

In [19]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


When we invoke this statement using `Session.execute()`, we now receive rows that have _individual elements per value_, each corresponding to a separate column or other SQL expression.

In [20]:
row = session.execute(select(User.name, User.fullname)).first()
print(row)

2022-09-25 09:38:09,274 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2022-09-25 09:38:09,275 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ()
('spongebob', 'Spongebob Squarepants')


The approaches can also be mixed, as below where we `SELECT` the _name attribute_ of the User entity as the `first element of the row`, and **combine** it with _full Address entities_ in the `second element`.

In [21]:
session.execute(
    select(User.name, Address).
    where(User.id==Address.id).
    order_by(Address.id)
).all()

2022-09-25 09:38:09,357 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.id ORDER BY address.id
2022-09-25 09:38:09,359 INFO sqlalchemy.engine.Engine [generated in 0.00164s] ()


[('spongebob', Address(id=1, email_address='spongebob@sqlalchemy.org')),
 ('sandy', Address(id=2, email_address='sandy@sqlalchemy.org')),
 ('patrick', Address(id=3, email_address='sandy@squirrelpower.org'))]

Approaches towards _selecting ORM entities and columns_ as well as common methods for _converting rows_ are discussed further at `Selecting ORM Entities and Attributes`.

##### Selecting from Labeled SQL Expressions

The `ColumnElement.label()` method as well as the _same-named method_ available on `ORM attributes` provides a _SQL label of a column or expression_, allowing it to have a **specific name** in a result set. This can be helpful when _referring to arbitrary SQL expressions in a result row by name_.

In [22]:
stmt = (
    select(
        ("Username: " + user_table.c.name).label("username"),
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")

2022-09-25 09:38:09,467 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-25 09:38:09,470 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2022-09-25 09:38:09,470 INFO sqlalchemy.engine.Engine [generated in 0.00349s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2022-09-25 09:38:09,472 INFO sqlalchemy.engine.Engine ROLLBACK
